# Universidad Autónoma de Nuevo León
## Facultad de Ciencias Fisico Matemáticas
### Maestría en Ciencia de Datos
#### Alumno: Francisco D. Treviño B.

### Tarea 2 (10 puntos). Operaciones con RDDs
### Obtener información de algún origen de datos (propio o de API)
### Convertir el origen de datos a RDD con pySpark
### Realizar alguna operación en el RDD, como estadísticas descriptivas básicas
### Subir la práctica a un repositorio público y etiquetarla claramente

# Instalación de Spark

## Se utilizó Docker para la instalación Spark usando una imagen, con el siguiente comando en Windows, según referencia en https://hub.docker.com/_/spark:
### docker pull spark


## Para correr Spark en Docker en modo interactivo se corre el siguiente comando:
### docker run -it spark:python3 /opt/spark/bin/pyspark

## Nota: no se utilizaron las siguientes instrucciones para instalar Spark.
#### !sudo apt update
#### !apt-get install openjdk-8-jdk-headless -qq > /dev/null
#### !wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#### !tar xf spark-3.72.1-bin-hadoop3.2.tgz

In [ ]:
!pip install -q findspark

In [ ]:
!pip install pyspark

In [ ]:
!pip install py4j

In [1]:
!pip install --upgrade pyspark

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Ejecutar el comando para iniciar Pyspark en un Notebook de Jupyter
# docker run -p 8888:8888 -p 4040:4040 -v "C:/Users/franc/Documents/UANL/MCD/07 Datos Masivos/Tareas/tarea-02":/home/jovyan/work --name pyspark-notebook -d -e PASSWORD=mcdbigdata


In [3]:
from pyspark import SparkConf
from pyspark import SparkContext as sc

In [4]:
from py4j.java_gateway import java_import

In [5]:
jvm = sc._jvm

In [7]:
java_import(jvm, "org.apache.spark.sql.api.python.*")

AttributeError: 'NoneType' object has no attribute '_gateway_client'

In [8]:
import os
import sys

import findspark
import pyspark

from pyspark.sql import DataFrame
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

### Dataframes de PySpark

In [9]:
# Inicializar sesión de Spark
from pyspark.sql import SparkSession

In [10]:
findspark.init()
findspark.find()

'C:\\Users\\franc\\AppData\\Roaming\\Python\\Python39\\site-packages\\pyspark'

In [ ]:
spark = SparkSession.builder \
    .appName("IntradayDataProcessing") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memoryOverhead", "2g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.cores", "2") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "120s") \
    .getOrCreate()

In [15]:
spark = SparkSession.builder \
    .appName("IntradayDataProcessing") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.network.timeout", "600s") \
    .config("spark.executor.heartbeatInterval", "120s") \
    .getOrCreate()

# Configurar la sesión de Spark con más memoria y recursos
spark = SparkSession.builder \
    .appName("Intraday_Data_Processing") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

In [16]:
# Paso 1: Convertir el DataFrame a RDD

# Lista de archivos CSV a cargar
# csv_file_names = ["intraday_bmv_a.txt", "intraday_bmv_b.txt", "intraday_bmv_c.txt", "intraday_bmv_d.txt"]
csv_file_names = ["intraday_bmv_a.txt"]

# Cargar los archivos CSV, especificando que están delimitados por TAB
df_list = []
for csv_file_name in csv_file_names:
    df = spark.read.csv(csv_file_name, sep="\t", header=True, inferSchema=True)
    df_list.append(df)

# Unir todos los DataFrames en uno solo (si es necesario)
combined_df = df_list[0]
for df in df_list[1:]:
    combined_df = combined_df.union(df)

In [17]:
df.printSchema()

root
 |-- trade_time: timestamp (nullable = true)
 |-- match_number: integer (nullable = true)
 |-- instrument_id: integer (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- volume: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- amount: double (nullable = true)
 |-- buyer_id: integer (nullable = true)
 |-- buyer_name: string (nullable = true)
 |-- seller_id: integer (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- auction_indicator: string (nullable = true)
 |-- settlement: string (nullable = true)
 |-- operation_type: string (nullable = true)
 |-- concertation_type: string (nullable = true)
 |-- price_setter: integer (nullable = true)
 |-- lot: integer (nullable = true)
 |-- chart: integer (nullable = true)
 |-- symbol: string (nullable = true)



In [25]:
combined_df = combined_df.repartition(10)

In [26]:
# Convertir el DataFrame a un RDD
rdd = combined_df.rdd

In [27]:
# Reparticionar los datos para tener más particiones y evitar sobrecarga en cada tarea
rdd = rdd.repartition(100)

In [28]:
spark.sparkContext.setLogLevel("DEBUG")  # O "INFO" para obtener más detalles

In [29]:
rdd.take(5)  # Muestra los primeros 5 registros del RDD

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 8.0 failed 1 times, most recent failure: Lost task 2.0 in stage 8.0 (TID 36) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(Inte

In [ ]:
combined_df.describe().show()

In [ ]:
combined_df.filter(combined_df.buyer_id.isin([149, 50])).orderBy(F.asc("trade_time")).limit(100).show()

In [ ]:
from pyspark.sql import functions as F

# Agrupar por 'buy_participant_id' y la fecha (día) extraída de 'trade_time', luego sumar 'executed_quantity'
combined_df.groupBy(F.to_date("trade_time").alias("trade_date"), "buyer_id") \
           .sum("volume") \
           .orderBy(F.desc("trade_date")) \
           .show(truncate=False)

In [ ]:
# Realizar estadísticas descriptivas básicas
price_rdd = rdd.map(lambda row: row["price"])

# Filtrar valores nulos
price_rdd = price_rdd.filter(lambda x: x is not None)

In [ ]:
# Calcular estadísticas descriptivas básicas
count = price_rdd.count()
mean = price_rdd.mean()
min_value = price_rdd.min()
max_value = price_rdd.max()
stddev = price_rdd.stdev()

# Mostrar los resultados
print(f"Count: {count}")
print(f"Mean: {mean}")
print(f"Min: {min_value}")
print(f"Max: {max_value}")
print(f"Standard Deviation: {stddev}")